In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import math
import os
import random

In [ ]:
# RANDOM GENERATOR:
class RandomNumberGenerator:
    def __init__(self, seedVaule = None):
        self.__seed = seedVaule

    def nextInt(self, low, high):
        m = 2147483647
        a = 16807
        b = 127773
        c = 2836
        
        k = int(self.__seed / b)
        self.__seed = a * (self.__seed % b) - k * c

        if self.__seed < 0:
            self.__seed = self.__seed + m

        value_0_1 = self.__seed
        value_0_1 =  value_0_1/m
        
        return low + int(math.floor(value_0_1 * (high - low + 1)))
    

    def nextFloat(self, low, high):
        low *= 100000
        high *= 100000
        val = self.nextInt(low, high) / 100000.0
        
        return val
    
SEED = 17
r = RandomNumberGenerator(SEED)

def get_input_data(n: int):
    S = 0
    d = []
    p = [[], [], []]

    for i in range(3):
        for j in range(n):
            p[i].append(r.nextInt(1, 99))
            S += p[i][j]
    for i in range(n):
        d.append(r.nextInt(math.floor(S/4), math.floor(S/2)))

    return d, p

In [ ]:
# zadanie 1:
def evaluate_permutation(permutation, p_times, due_dates):
    m = 3
    n = len(permutation)
    C = [[0]*n for _ in range(m)]
    
    first_job = permutation[0]
    C[0][0] = p_times[0][first_job]

    for j in range(1, n):
        job = permutation[j]
        C[0][j] = C[0][j-1] + p_times[0][job]

    for i in range(1, m):
        job0 = permutation[0]
        C[i][0] = C[i-1][0] + p_times[i][job0]

    for i in range(1, m):
        for j in range(1, n):
            job = permutation[j]
            C[i][j] = max(C[i-1][j], C[i][j-1]) + p_times[i][job]
    
    f1 = max(C[m-1]) 
    
    f2 = sum(C[m-1][j] for j in range(n))
    
    return f1, f2


def random_neighbor(permutation):
    n = len(permutation)
    i, j = random.sample(range(n), 2)
    neighbor = permutation.copy()
    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor


def dominates(a, b):
    return (a[0] <= b[0] and a[1] <= b[1]) and (a[0] < b[0] or a[1] < b[1])


def pareto_front(solutions):
    front = []
    for perm_a, obj_a in solutions:
        dominated_flag = False
        for perm_b, obj_b in solutions:
            if perm_a == perm_b:
                continue
            if dominates(obj_b, obj_a):
                dominated_flag = True
                break
        if not dominated_flag:
            front.append((perm_a, obj_a))
    return front


def simulated_annealing_multi(
    p_times,
    due_dates,
    max_iter=1000,
    init_temp=1.0,
    cooling_rate=0.995,
    seed=None
):
    if seed is not None:
        random.seed(seed)
    
    n = len(due_dates)

    current_perm = list(range(n))
    random.shuffle(current_perm)
    current_obj = evaluate_permutation(current_perm, p_times, due_dates)
    
    P = [(current_perm.copy(), current_obj)]
    
    temp = init_temp
    
    for it in range(1, max_iter+1):
        neighbor = random_neighbor(current_perm)
        obj_neighbor = evaluate_permutation(neighbor, p_times, due_dates)
        
        if dominates(obj_neighbor, current_obj):
            current_perm = neighbor
            current_obj = obj_neighbor
            P.append((current_perm.copy(), current_obj))
        else:
            delta = (obj_neighbor[0] + obj_neighbor[1]) - (current_obj[0] + current_obj[1])

            prob = min(1.0,  pow(2.718281828, -delta / temp) )
            if random.random() < prob:
                current_perm = neighbor
                current_obj = obj_neighbor
                P.append((current_perm.copy(), current_obj))
        
        temp *= cooling_rate
    
    F = pareto_front(P)
    
    return P, F

In [ ]:
d, p = get_input_data(10)
max_iter = 1000
init_temp = 1.0
cooling_rate = 0.995
seed = SEED
P, F = simulated_annealing_multi(p, d, max_iter=max_iter, init_temp=init_temp, cooling_rate=cooling_rate, seed=seed)
